In [1]:
from datasets import load_dataset
import datasets
#dataset = load_dataset("yelp_review_full")
dataset = datasets.load_from_disk("/mnt/workspace/data/yelp_review_full_data")

/opt/conda/envs/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [3]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [4]:
show_random_elements(dataset["train"])

,label,text
0,2 star,"Their curried vegetables are pretty bland. I would definitely recommend Husnu's (right next door) over Kabul for any vegans. The vegan food is tastier there, and you can always eat the soup or salad that comes with your meal. The salad dressing at Kabul has eggs in it, and when I went, they didn't have a vegan soup. The service at Kabul was good, though."
1,1 star,"Not impressed. We had VIP access but that wasn't anything very impressive. Not a fan of the crowd or the music. Some people there were extra rude and snotty. I'm not sure if it was just the night we were there or its this way all the time, but I would definitely not go back."
2,4 stars,"Food was great!! Found this place by accident while looking for food trucks. Lots on the menu to choose from,from Asian to Euro sandwiches, churros,steamed buns, and fried chicken. Baguette reminded me of being back on Germany. \nOne bothersome item os the $20 limit to not get charged a fee."
3,2 star,"I'm not going to lie but even Fry's sushi is better than what I had tonight. I think my dad put it the best way: This was made for Americans. I'm not trying to be racist but coming from the bay area, I've definitely had better. The portion sizes were amazing but it just wasn't appetizing. I definitely won't be back but nice try Arizona sushi place"
4,3 stars,"The 3 stars are for the food ONLY!\nGet it to-go or suffer through the wait lines on any sporting event day because they don't take reservations ahead of time, and the shoddy excuse for service. Oh did I mention you REALLY shouldn't order water because it takes like dirt and smells like chemicals."
5,1 star,"This is for breakfast only. The lunch looks good. \nI got a Denver omlet. A little pricey at almost $12 for the omlet, biscut and McDonald's type deep fried hash browns. So salty I could only eat a little and leave. \nWon't eat here again for breakfast. Maybe lunch is better."
6,4 stars,"YAY Paris Buffet!!! This is the one thing I was so excited about when I agreed to go to Vegas with the family--breakfast at the Paris Buffet. The price is very reasonable--$15.99. And, once again, because of my parents and their rewards card, we didn't have to stand in an hour-long line...\n\nNow, I am a huge fan of the breakfast meats. Any variety, and shape, and size. Almost every station had at least one type of breakfast meat. Better yet, one entire station was all breakfast meats. I was overwhelmed!! I took a huge breath, then made the smart choice to sample some foods before I committed and took a giant portion. One of my faves was a beef and turkey sausage with sweet potato and maple syrup. It's got it all in one casing!! So yummy. I also greatly enjoyed the poached eggs Basquaise. It was a poached egg with a really delicious, somewhat spicy tomato sauce. My breakfast dessert was a crepe with Nutella and blueberries--YUMMO!! And it's always nice when the coffee's good. That way, when you bundle up to face the freezing cold of winter-time Vegas, you've got some good caffeine in your veins. Cheers to the Paris Buffet!!"
7,1 star,"As a local I'll never visit this joint again. Burger was decent, under cooked from what I ordered but I'll deal with a slight under cook although not sure I should at $12 for a burger + $5 for fries b/c they are not included in the meal. $17 burger, really? mine is better at home and I have a community grill, not the, \""only coal oven in the country\"", like the grind does. \n\nHappy hr charges $3 for sprite on well drinks, don't be fooled by a BS happy hr special. Not a bad value if you are just having beer but leave the well drinks for home. \n\nOverall this place clearly thinks they are more than they really are. My suggestion go down the road and try either the Vig or Arcadia Tavern, way more value for the $ unless you are trying to impress the newest blonde to move from Scottsdale to Arcadia/Biltmore area."
8,4 stars,"From my wife: \""I just got married last year. I bought my dress in 

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/mnt/workspace/model/bert_base_cased",local_files_only=True)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [6]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,4 stars,"This place overlooks the US National Whitewater Center's main activity area. It is an absolutely gorgeous view come sunset and perfect on Thursday nights during the summer when they have free live entertainment. I have been to the USNWC before but never ate at this place. It did not disappoint, we had quick seating and service. They had a nice beer selection---not overbearing like some other places and the prices weren't too high. \n\nMy group had an appetizer of fried pickles with ranch dipping sauce which was great (you really cant go wrong with that) \n\nI had the Apple Walnut Chicken Salad -Marinated grilled chicken atop spring mix with apples, crumbled bleu cheese and candied walnuts with Balsamic Vinaigrette \n\nFantastic salad! It was very refreshing and the candied walnuts added a nice sweet touch to contrast the apples. And I am all for any kind of crumbled cheese! I was looking at the hummus plate but needed something cool and refreshing and this hit the spot! \n\nI definitely recommend this place, the service was good, food was great and you cannot beat the location. Go on a Thursday or Saturday night during the summer and there is live music and you can hang around after and just chill. So happy Charlotte has this place!","[101, 1188, 1282, 1166, 21163, 1116, 1103, 1646, 1305, 2061, 4669, 1945, 112, 188, 1514, 3246, 1298, 119, 1135, 1110, 1126, 7284, 10144, 2458, 1435, 16855, 1105, 3264, 1113, 9170, 6823, 1219, 1103, 2247, 1165, 1152, 1138, 1714, 1686, 5936, 119, 146, 1138, 1151, 1106, 1103, 1646, 2249, 16697, 1196, 1133, 1309, 8756, 1120, 1142, 1282, 119, 1135, 1225, 1136, 4267, 3202, 8661, 21506, 117, 1195, 1125, 3613, 11051, 1105, 1555, 119, 1220, 1125, 170, 3505, 5298, 4557, 118, 118, 118, 1136, 1166, 3962, 10832, 1176, 1199, 1168, 2844, 1105, 1103, 7352, 3920, 112, 189, 1315, 1344, 119, 165, 183, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


In [7]:
import numpy as np
import evaluate

metric = evaluate.load("/mnt/workspace/evaluate/metrics/accuracy")

2024-01-15 19:12:12.828314: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 19:12:13.751619: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("/mnt/workspace/models/bert-base-cased/finetuned-trainer",local_files_only=True, num_labels=5)

In [11]:
from transformers import TrainingArguments,Trainer

model_dir = "models_full/bert-base-cased"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=1000)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

Detected kernel version 4.19.24, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [21]:
! nvidia-smi

Thu Jan  4 17:48:13 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:08.0 Off |                    0 |
| N/A   34C    P0    56W / 300W |   1534MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
trainer.train()

Step,Training Loss
1000,1.096800
2000,1.012700
3000,0.977300
4000,0.964600
5000,0.949400
6000,0.929500
7000,0.933800
8000,0.916200
9000,0.921200
10000,0.925700


Checkpoint destination directory models_full/bert-base-cased/test_trainer/checkpoint-9000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory models_full/bert-base-cased/test_trainer/checkpoint-9500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


KeyboardInterrupt: 

In [10]:
from transformers import AutoModelForSequenceClassification,TrainingArguments,Trainer
model_dir = "models_full/bert-base-cased"
model2 = AutoModelForSequenceClassification.from_pretrained("/mnt/workspace/models_full/bert-base-cased/test_trainer/checkpoint-12000",local_files_only=True, num_labels=5)
training_args2 = TrainingArguments(output_dir=f"{model_dir}/test_trainer2",
                                  logging_dir=f"{model_dir}/test_trainer2/runs",
                                   save_total_limit=10
                                  )

In [11]:
trainer2 = Trainer(
    model=model2,
    args=training_args2,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

Detected kernel version 4.19.24, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [18]:
import torch
torch.cuda.empty_cache()

In [12]:
trainer2.train()

Step,Training Loss
500,0.868800
1000,0.831200
1500,0.822100
2000,0.844000
2500,0.840400
3000,0.807800
3500,0.804200
4000,0.808000
4500,0.817300
5000,0.796500


TrainOutput(global_step=243750, training_loss=0.8492603022961739, metrics={'train_runtime': 60574.1232, 'train_samples_per_second': 32.192, 'train_steps_per_second': 4.024, 'total_flos': 5.130803778048e+17, 'train_loss': 0.8492603022961739, 'epoch': 3.0})

In [13]:
test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(10000))

In [14]:
trainer2.evaluate(test_dataset)

{'eval_loss': 0.8118643760681152,
 'eval_accuracy': 0.6595,
 'eval_runtime': 105.2378,
 'eval_samples_per_second': 95.023,
 'eval_steps_per_second': 11.878,
 'epoch': 3.0}

In [17]:
trainer2.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.8135793805122375,
 'eval_accuracy': 0.65844,
 'eval_runtime': 526.3094,
 'eval_samples_per_second': 95.001,
 'eval_steps_per_second': 11.875,
 'epoch': 3.0}

In [15]:
trainer2.save_model(f"{model_dir}/finetuned-trainer-full")

In [16]:
trainer2.save_state()